In [1]:
#!pip install chesslab

In [2]:
import numpy as np
from chesslab.utils import load_pkl
from chesslab.training_tf import fitting
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [3]:
example=1
lr = 0.1
epochs=30
batch_size = 128
test_percent=0.1

path = 'D:/database/ccrl/'
name_data='ccrl_states_elo2.pkl'
name_labels='ccrl_results_elo2.pkl'
save_name='./tmp/tf_weights.0'

optim = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=lr)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [4]:
download=False
if download:
    from chesslab.utils import download_7z
    path='./'
    file_id = '1MFHFz_rxNziYSeN-9ruwnRiYskd0_9ss'
    download_7z(file_id,path)

In [5]:
encoding_1={
    '.':np.array([ 0, 0, 0],dtype=np.float32),
    'p':np.array([ 0, 0, 1],dtype=np.float32),
    'P':np.array([ 0, 0,-1],dtype=np.float32),
    'b':np.array([ 0, 1, 0],dtype=np.float32),
    'B':np.array([ 0,-1, 0],dtype=np.float32),
    'n':np.array([ 1, 0, 0],dtype=np.float32),
    'N':np.array([-1, 0, 0],dtype=np.float32),
    'r':np.array([ 0, 1, 1],dtype=np.float32),
    'R':np.array([ 0,-1,-1],dtype=np.float32),
    'q':np.array([ 1, 0, 1],dtype=np.float32),
    'Q':np.array([-1, 0,-1],dtype=np.float32),
    'k':np.array([ 1, 1, 0],dtype=np.float32),
    'K':np.array([-1,-1, 0],dtype=np.float32)
}

encoding_2={
    '.':np.array([0,0,0,0],dtype=np.float32),
    'p':np.array([1,0,0,0],dtype=np.float32),
    'P':np.array([0,0,0,1],dtype=np.float32),
    'b':np.array([0,1,0,0],dtype=np.float32),
    'B':np.array([0,0,1,0],dtype=np.float32),
    'n':np.array([1,1,0,0],dtype=np.float32),
    'N':np.array([0,0,1,1],dtype=np.float32),
    'r':np.array([1,0,1,0],dtype=np.float32),
    'R':np.array([0,1,0,1],dtype=np.float32),
    'q':np.array([1,0,0,1],dtype=np.float32),
    'Q':np.array([0,1,1,0],dtype=np.float32),
    'k':np.array([1,1,1,0],dtype=np.float32),
    'K':np.array([0,1,1,1],dtype=np.float32)
}


In [6]:
class Model_1():

    def __init__(self,
                   n_classes=2):
        initializer = tf.keras.initializers.GlorotNormal()
        self.hw=[]
        self.hb=[]

        self.hw.append( tf.Variable(initializer(shape=(7,7,3,32),dtype=np.float32),name="hl1weigths",dtype="float32") )
        self.hb.append( tf.Variable(np.zeros(32,dtype=np.float32),name="hl1bias",dtype="float32") )
        #8x8x32
        self.hw.append( tf.Variable(initializer(shape=(5,5,32,64),dtype=np.float32),name="hl2weigths",dtype="float32"))
        self.hb.append( tf.Variable(np.zeros(64,dtype=np.float32),name="hl2bias",dtype="float32"))
        #8x8x64
        self.hw.append( tf.Variable(initializer(shape=(3,3,64,128),dtype=np.float32),name="hl3weigths",dtype="float32"))
        self.hb.append( tf.Variable(np.zeros(128,dtype=np.float32),name="hl3bias",dtype="float32"))
        #8x8x128
        self.hw.append( tf.Variable(initializer(shape=(8*8*128,256),dtype=np.float32),name="hl4weigths",dtype="float32"))
        self.hb.append( tf.Variable(np.zeros(256,dtype=np.float32),name="hl4bias",dtype="float32"))

        self.hw.append( tf.Variable(initializer(shape=(256, n_classes),dtype=np.float32),name="outweigths",dtype="float32"))
        self.hb.append( tf.Variable(np.zeros(n_classes,dtype=np.float32),name="outbias",dtype="float32"))

        self.trainable_variables = []
        for i in range(len(self.hw)):
            self.trainable_variables.append(self.hw[i])    
            self.trainable_variables.append(self.hb[i])

    def __call__(self,x): 

        # Declarando la arquitectura
        out = tf.cast(x, tf.float32)
        out = tf.reshape(out, shape=[-1, 8, 8, 3])

        layer=0
        out = tf.add(out, 1e-8)
        out = tf.nn.conv2d(out,self.hw[layer], strides=[1,1,1,1], padding='SAME')  
        out = tf.add(out, self.hb[layer])
        out = tf.nn.elu(out)
              #8*8*32
        layer+=1
        out = tf.nn.conv2d(out,self.hw[layer], strides=[1,1,1,1], padding='SAME')  
        out = tf.add(out, self.hb[layer])
        out = tf.nn.elu(out)
              #8*8*64
        layer+=1  
        out = tf.nn.conv2d(out,self.hw[layer], strides=[1,1,1,1], padding='SAME')  
        out = tf.add(out, self.hb[layer])
        out = tf.nn.elu(out)
              #8*8*128
        layer+=1
        out =  tf.reshape(out,[-1, 8*8*128])
        out =  tf.matmul(out,self.hw[layer])
        out = tf.add(out, self.hb[layer])
        out = tf.nn.elu(out)

        layer+=1
        out =  tf.matmul(out,self.hw[layer])
        out = tf.add(out, self.hb[layer])

        return out
    

class Model_2():

    def __init__(self,
                   n_classes=2):
        initializer = tf.keras.initializers.GlorotNormal()
        self.hw=[]
        self.hb=[]

        self.hw.append( tf.Variable(initializer(shape=(7,7,4,32),dtype=np.float32),name="hl1weigths",dtype="float32") )
        self.hb.append( tf.Variable(np.zeros(32,dtype=np.float32),name="hl1bias",dtype="float32") )
        #8x8x32
        self.hw.append( tf.Variable(initializer(shape=(5,5,32,64),dtype=np.float32),name="hl2weigths",dtype="float32"))
        self.hb.append( tf.Variable(np.zeros(64,dtype=np.float32),name="hl2bias",dtype="float32"))
        #8x8x64
        self.hw.append( tf.Variable(initializer(shape=(3,3,64,128),dtype=np.float32),name="hl3weigths",dtype="float32"))
        self.hb.append( tf.Variable(np.zeros(128,dtype=np.float32),name="hl3bias",dtype="float32"))
        #8x8x128
        self.hw.append( tf.Variable(initializer(shape=(8*8*128,256),dtype=np.float32),name="hl4weigths",dtype="float32"))
        self.hb.append( tf.Variable(np.zeros(256,dtype=np.float32),name="hl4bias",dtype="float32"))

        self.hw.append( tf.Variable(initializer(shape=(256, n_classes),dtype=np.float32),name="outweigths",dtype="float32"))
        self.hb.append( tf.Variable(np.zeros(n_classes,dtype=np.float32),name="outbias",dtype="float32"))

        self.trainable_variables = []
        for i in range(len(self.hw)):
            self.trainable_variables.append(self.hw[i])    
            self.trainable_variables.append(self.hb[i])

    def __call__(self,x): 

        out = tf.cast(x, tf.float32)
        out = tf.reshape(out, shape=[-1, 8, 8, 4])

        layer=0
        out = tf.nn.conv2d(out,self.hw[layer], strides=[1,1,1,1], padding='SAME')  
        out = tf.add(out, self.hb[layer])
        out = tf.nn.relu(out)
              #8*8*32
        layer+=1
        out = tf.nn.conv2d(out,self.hw[layer], strides=[1,1,1,1], padding='SAME')  
        out = tf.add(out, self.hb[layer])
        out = tf.nn.relu(out)
              #8*8*64
        layer+=1  
        out = tf.nn.conv2d(out,self.hw[layer], strides=[1,1,1,1], padding='SAME')  
        out = tf.add(out, self.hb[layer])
        out = tf.nn.elu(out)
              #8*8*128
        layer+=1
        out =  tf.reshape(out,[-1, 8*8*128])
        out =  tf.matmul(out,self.hw[layer])
        out = tf.add(out, self.hb[layer])
        out = tf.nn.relu(out)

        layer+=1
        out =  tf.matmul(out,self.hw[layer])
        out = tf.add(out, self.hb[layer])

        return out

In [7]:
np.random.seed(0)
tf.random.set_seed(0)

x_data = load_pkl(path+name_data)
y_data = load_pkl(path+name_labels)[:,1] #Nota: pasa de onehot a logits

print(x_data.shape)
print(y_data.shape)

x_train, x_test, y_train, y_test = train_test_split(
    x_data, y_data, test_size = test_percent, random_state = 0, shuffle = True)

del x_data
del y_data

(4829155, 64)
(4829155,)


In [8]:
if example==1:
    model = Model_1()
    encoding=encoding_1
else:
    model = Model_2()
    encoding=encoding_2


In [9]:
fitting(epochs=epochs,
        x_train=x_train,
        y_train=y_train,
        x_test=x_test,
        y_test=y_test,
        model=model,
        optimizer=optim,
        batch_size=batch_size,
        lr=lr,
        loss_fn=loss_fn,
        save_name=save_name,
        encoding=encoding)

2021-10-25 15:10:56


Epoch: 01/30 | time: 177s = 3.0m | train loss: 0.5441 | train acc: 0.7118 | test loss: 0.5158 | test acc: 0.7322

Epoch: 02/30 | time: 172s = 2.9m | train loss: 0.5029 | train acc: 0.7411 | test loss: 0.4866 | test acc: 0.7521

Epoch: 03/30 | time: 175s = 2.9m | train loss: 0.4717 | train acc: 0.7619 | test loss: 0.4512 | test acc: 0.7755

Epoch: 04/30 | time: 174s = 2.9m | train loss: 0.4477 | train acc: 0.7774 | test loss: 0.4648 | test acc: 0.7591

...

Epoch: 26/30 | time: 179s = 3.0m | train loss: 0.1591 | train acc: 0.9415 | test loss: 0.3643 | test acc: 0.8891

Epoch: 27/30 | time: 178s = 3.0m | train loss: 0.1595 | train acc: 0.9417 | test loss: 0.3773 | test acc: 0.8908

Epoch: 28/30 | time: 178s = 3.0m | train loss: 0.1606 | train acc: 0.9416 | test loss: 0.3749 | test acc: 0.8932

Epoch: 29/30 | time: 177s = 2.9m | train loss: 0.1595 | train acc: 0.9421 | test loss: 0.3665 | test acc: 0.8895

Epoch: 30/30 | time: 176s = 2.9m | train loss: 0.1598 | train acc: 0.9423 | test loss: 0.3495 | test acc: 0.8922

fitting(epochs=1,
        x_train=x_train,
        y_train=y_train,
        x_test=x_test,
        y_test=y_test,
        model= model, 
        load_name='tmp/tf_weights.0.6.h5',
        save_name=save_name,)